In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [3]:
#Number of features (i.e. input units, vocabulary size)
features_n = 300
#Number of output units (i.e. categories)
cats_n = 4
#Number of hidden units
hidden_units_n = 100

#Learning rate, used for testing
learning_rate = 0.001
#Batch size, used for testing
batch_size = 30
#Number of epochs,used for testing
epochs_n = 10
#Reporing frequency, used for testing
display_freq = 100

In [6]:
tf.reset_default_graph()
#
# Create TF graph
#
#
with tf.Session() as session:

    print(f'Graph created successfully')

    #Input
    inputs = tf.placeholder(tf.float32, shape=[None, features_n], name='inputs')
    #Output
    outputs = tf.placeholder(tf.float32, shape=[None, cats_n], name='outputs')

    #Initialization of weigths to hidden layer
    hidden_init = tf.truncated_normal_initializer(stddev=0.01)

    #Input->hidden weights
    input_to_hidden_w = tf.get_variable(
         "hidden_weights", dtype=tf.float32, shape=[features_n, hidden_units_n], initializer=hidden_init)

    
    #Initialization of bias to hidden layer
    hidden_bias_init = tf.constant(0., shape=[hidden_units_n], dtype=tf.float32)

    #Bias->hidden weights
    bias_to_hidden_w = tf.get_variable("hidden_bias", dtype=tf.float32, initializer=hidden_bias_init)

    #Hidden layer, activation function is sigmoid
    hidden_layer = tf.matmul(inputs, input_to_hidden_w) + bias_to_hidden_w
    hidden_layer = tf.nn.sigmoid(hidden_layer)

    
    #Initialization of weigths to output layer
    output_init = tf.truncated_normal_initializer(stddev=0.01)

    #Hidden->output weights
    hidden_to_output_w = tf.get_variable(
         "output_weights", dtype=tf.float32, shape=[hidden_units_n, cats_n], initializer=output_init)

    #Initialization of bias to output layer
    output_bias_init = tf.constant(0., shape=[cats_n], dtype=tf.float32)

    #Bias->output weights
    bias_to_output_w = tf.get_variable("output_bias", dtype=tf.float32, initializer=output_bias_init)

    #Output layer, no activation (i.e. logits)
    output_layer = tf.matmul(hidden_layer, hidden_to_output_w) + bias_to_output_w

    #Loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=outputs, logits=output_layer), name='loss')

    #Optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='Adam').minimize(loss)

    #Accuracy per trial
    correct_prediction = tf.equal(tf.argmax(output_layer, 1), tf.argmax(outputs, 1), name='correct_prediction')

    #Overall accuracy/by batch
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

    #Model predictions
    cls_prediction = tf.argmax(output_layer, axis=1, name='predictions')

    #TF variables initialization
    init = tf.global_variables_initializer()
    
    #Save TF graph
    tf.train.Saver()    
    tf.train.write_graph(session.graph, "/home/i/projects/johnsnow/logs", "/home/i/projects/johnsnow/mymodel.pb", False)
    

Graph created successfully


In [10]:
#Some aux fuctions from the internet

def randomize(x, y):
    """ Randomizes the order of data samples and their corresponding labels"""
    permutation = np.random.permutation(y.shape[0])
    shuffled_x = x[permutation, :]
    shuffled_y = y[permutation]
    return shuffled_x, shuffled_y

def get_next_batch(x, y, start, end):
    x_batch = x[start:end]
    y_batch = y[start:end]
    return x_batch, y_batch

In [11]:
#
# Generate dummy train/validation set for testing the model
#
x_train = np.zeros(shape=(batch_size * 1000, features_n))
y_train = np.zeros(shape=(batch_size * 1000, cats_n))

x_train[:batch_size * 250] = np.random.normal(0, 1, size=(batch_size * 250, features_n))
x_train[batch_size * 250:batch_size * 500] = np.random.normal(1, 1, size=(batch_size * 250, features_n))
x_train[batch_size * 500:batch_size * 750] = np.random.normal(2, 1, size=(batch_size * 250, features_n))
x_train[batch_size * 750:] = np.random.normal(3, 1, size=(batch_size * 250, features_n))

y_train[:batch_size * 250, 0] = 1
y_train[batch_size * 250:batch_size * 500, 1] = 1
y_train[batch_size * 500:batch_size * 750, 2] = 1
y_train[batch_size * 750:, 3] = 1

x_train, y_train = randomize(x_train, y_train)

x_val = x_train[batch_size * 900:,]
y_val = y_train[batch_size * 900:,]

x_train = x_train[:batch_size * 900,]
y_train = y_train[:batch_size * 900,]


In [12]:
#
# Testing the model on dummy data
#

sess = tf.InteractiveSession()
sess.run(init)

# Number of training iterations in each epoch

num_tr_iter = int(len(y_train) / batch_size)

for epoch in range(epochs_n):
    print('Training epoch: {}'.format(epoch + 1))
    # Randomly shuffle the training data at the beginning of each epoch 
    x_train, y_train = randomize(x_train, y_train)
    
    for iteration in range(num_tr_iter):
        start = iteration * batch_size
        end = (iteration + 1) * batch_size
        x_batch, y_batch = get_next_batch(x_train, y_train, start, end)

        # Run optimization op (backprop)
        feed_dict_batch = {inputs: x_batch, outputs: y_batch}
        sess.run(optimizer, feed_dict=feed_dict_batch)

        if iteration % display_freq == 0:
            # Calculate and display the batch loss and accuracy
            loss_batch, acc_batch = sess.run([loss, accuracy],
                                             feed_dict=feed_dict_batch)

            print("iter {0:3d}:\t Loss={1:.2f},\tTraining Accuracy={2:.01%}".
                  format(iteration, loss_batch, acc_batch))

    # Run validation after every epoch
    feed_dict_valid = {inputs: x_val, outputs: y_val}
    loss_valid, acc_valid = sess.run([loss, accuracy], feed_dict=feed_dict_valid)
    print('---------------------------------------------------------')
    print("Epoch: {0}, validation loss: {1:.2f}, validation accuracy: {2:.01%}".
          format(epoch + 1, loss_valid, acc_valid))
    print('---------------------------------------------------------')

sess.close()

/home/i/penv/johnsnow/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Training epoch: 1
iter   0:	 Loss=1.35,	Training Accuracy=43.3%
iter 100:	 Loss=0.71,	Training Accuracy=100.0%
iter 200:	 Loss=0.63,	Training Accuracy=86.7%
iter 300:	 Loss=0.52,	Training Accuracy=100.0%
iter 400:	 Loss=0.37,	Training Accuracy=100.0%
iter 500:	 Loss=0.39,	Training Accuracy=100.0%
iter 600:	 Loss=0.30,	Training Accuracy=100.0%
iter 700:	 Loss=0.27,	Training Accuracy=100.0%
iter 800:	 Loss=0.24,	Training Accuracy=100.0%
---------------------------------------------------------
Epoch: 1, validation loss: 0.25, validation accuracy: 99.3%
---------------------------------------------------------
Training epoch: 2
iter   0:	 Loss=0.26,	Training Accuracy=100.0%
iter 100:	 Loss=0.14,	Training Accuracy=100.0%
iter 200:	 Loss=0.16,	Training Accuracy=100.0%
iter 300:	 Loss=0.12,	Training Accuracy=100.0%
iter 400:	 Loss=0.12,	Training Accuracy=100.0%
iter 500:	 Loss=0.11,	Training Accuracy=100.0%
iter 600:	 Loss=0.09,	Training Accuracy=100.0%
iter 700:	 Loss=0.06,	Training Accurac

In [13]:
#Let's look at the graph
session.graph.get_operations()

[<tf.Operation 'inputs' type=Placeholder>,
 <tf.Operation 'outputs' type=Placeholder>,
 <tf.Operation 'hidden_weights/Initializer/truncated_normal/shape' type=Const>,
 <tf.Operation 'hidden_weights/Initializer/truncated_normal/mean' type=Const>,
 <tf.Operation 'hidden_weights/Initializer/truncated_normal/stddev' type=Const>,
 <tf.Operation 'hidden_weights/Initializer/truncated_normal/TruncatedNormal' type=TruncatedNormal>,
 <tf.Operation 'hidden_weights/Initializer/truncated_normal/mul' type=Mul>,
 <tf.Operation 'hidden_weights/Initializer/truncated_normal' type=Add>,
 <tf.Operation 'hidden_weights' type=VariableV2>,
 <tf.Operation 'hidden_weights/Assign' type=Assign>,
 <tf.Operation 'hidden_weights/read' type=Identity>,
 <tf.Operation 'Const' type=Const>,
 <tf.Operation 'hidden_bias' type=VariableV2>,
 <tf.Operation 'hidden_bias/Assign' type=Assign>,
 <tf.Operation 'hidden_bias/read' type=Identity>,
 <tf.Operation 'MatMul' type=MatMul>,
 <tf.Operation 'add' type=AddV2>,
 <tf.Operation